In [ ]:
import ruckus
import numpy as np
import matplotlib.pyplot as plt

# Predictive states for hidden Markov models with ruckus

## Overview

- Using convolution in `ruckus`
- Working with categorical sequences
- Explicitly reconstructing predictions
- Using cross-validation to determine dimension

## Important terms

* HMM (or *hidden Markov model*): graphical model for discrete time-series data. Typically defined by a certain number of states $S$, and finite set of categorical observations $X$, and to each $x$ in $X$, an $S\times S$ matrix $T^{(x)} = \mathrm{Pr}(s',x|s)$ that describes the probability of generating the symbol $x$ and transitioning $s\rightarrow s'$.
* Predictive state: For a time series $(x_t)$, the predictive state at time $t$ the the probability distribution over future observations conditioned on previous observations: $\mathrm{Pr}(x_1,x_2,\dots|x_0,x_{-1},\dots)$. Typically calculated over finite windows: $\mathrm{Pr}(x_1,\dots,x_{L_f}|x_0,\dots,x_{-L_p+1})$. Though such probability distributions live in high-dimensional spaces, they often only use a small number of those dimensions.
* RKHS (or, *reproducing kernel Hilbert space*): A (typically high-dimensional) vector space $H$ with feature functions $\phi:X\rightarrow H$ that embed points $x$ from a data space $X$ as vectors in $H$. The kernel is given by the inner product $k(x,y) = \left<\phi(x),\phi(y)\right>$. RKHS's can be learned to use function representations: the representation $\hat{f}$ of $f$ is the vector $\hat{f}\in H$ for which $\left<\hat{f},\phi(x)\right> = f(x)$.

## Reconstructing predictive states: discrete method

We're going to generate a sequence using a particular hidden Markov model. You will be able to specify which using the key to the `transition_matrices` dictionary. Each option has unique features:

1. `golden` generates the Golden Mean process, which is a simple two-state Markov process (each symbol depends only on previous symbols). Consequently, there are only 2 distinct predictive states, which span 2 dimensions.
2. `even` generates the Even process, which contains sequences of 0's and 1's where 1's only appear in contiguous blocks of even size. Though the Markov order of this process is infinite (that is, dependence on the past can be arbitrarily long-range), there are still only 2 distinct predictive states, which span 2 dimensions.
3. `nemo` generates the Nemo process, which also has infinite Markov order but which has 3 distinct predictive states, spanning 3 dimensions.
4. `sns` generates the Simple Nonunifilar Source, which has an *infinite number* of distinct predictive states---though they still only span 2 dimensions.

Our goal in this notebook will be to use `ruckus` and `scikit-learn` to (1) determine the function from past observations to predictive states, (2) determine the minimal number of dimensions needed to express the predictive states, and (3) use the sequence of predictive states to determine a GHMM which reproduces the process.

The other option you may set below is the number of datapoints to generate. The default is current $2^{12} = 4096$. Feel free to make it larger, but be warned that this will increase the time it takes things to run!

In [ ]:
from tutorial_utils import transition_matrices, gen_hmm

data_Ts = transition_matrices['sns'] # <-- Insert option here!
X = gen_hmm(data_Ts,num=2**16)
N = X.shape[0]
print('N = ', X.shape[0])

For the first two problems we will take two different tracks. One will use `ruckus.OneHotRKHS` to construct probability vectors. Manipulating the one-hot vectors directly will allow us to compute conditional and joint probability vectors. In the next section, we will examine how this process can be generalized using other types of embeddings, and we shall examine if this provides any advantage.

## Building the embedding

The first step is to define our window length for our pasts and futures (we have set `L = 3`, but feel free to toggle - things start taking appreciable time to run around `L = 10`) and run a convolution over the data. This convolution will use a window of size `2*L+1` (pasts will be taken at length `L+1` and futures at length `L`, for reasons we'll explain later). The window will be processed by a `ProductRKHS` made up from two factors. The first handles pasts, and takes the first `L+1` items in the window and embeds each in their own `OneHotRKHS`. The second handles futures, and embeds each of the next `L` items in their own `OneHotRKHS`.

In [ ]:
L = 10

past_onehot = ruckus.OneHotRKHS(
    take = np.arange(L+1),
)
fut_onehot = ruckus.OneHotRKHS(
    take = np.arange(L+1,2*L+1),
)
prod_space = ruckus.ProductRKHS([past_onehot,fut_onehot])

conv_net = ruckus.convolution.ConvolutionalRKHS(
    window_shape=(2*L+1,),
    rkhs = prod_space
)

conv_net.fit(X)

The output of this cell tells us something about what we've just constructed: two product spaces, one for *past* data and one for *future* data. The past RKHS does a one-hot embedding which contains exactly one dimension for each unique past word. The future RKHS does the same for future words.

Note that `prod_space`, in its fully fitted form, can be accessed directly from the variable name `prod_space` or as `conv_net.rkhs`. Similarly, the variable names `past_onehot` and `fut_onehot` now refer to the *fitted* one-hot embeddings, as do `prod_space.factors[0]` and `prod_space.factors[1]`, respectively.

Each of the fitted components of our simple network contains a copy of the data used to fit it, in the attribute `X_fit_`. We can recall this and `transform` it to explicitly extract our one-hot embeddings:

In [ ]:
past_vecs = past_onehot.transform(past_onehot.X_fit_)
fut_vecs = fut_onehot.transform(fut_onehot.X_fit_)

Let's see, for example, what `fut_vecs` looks like:

In [ ]:
print(fut_vecs.shape)
print(fut_vecs)

We see that for each of our `N-2*L` windows, there is a vector (if you kept `L=3`, it should be of length `7`). The length of this vector corresponds to the number of unique length-`L` futures observed by the convolutional window. Each vector is made mostly of `0`s, with a single `1` in the component corresponding to the unique future word that was observed at that window.

We can see how the unique words are indexed by examining the `alphabet_` attribute:

In [ ]:
fut_onehot.alphabet_

So, `fut_vecs[t]` is a vector which has a `1` at index `j`, where `j` is the `fut_onehot.alphabet_` index corresponding to the future word observed at time `t`.

## Reconstructing predictive states

The main convenience of one-hot vectors is that they make *counting* really, really easy. For instance, if we want to find the joint counts of past word `past_onehot.alphabet_[i]` and future word `past_onehot.alphabet_[j]`, we can find those as the `i,j` entry in the matrix `past_vecs.T@fut_vecs`:

In [ ]:
joint_counts = past_vecs.T@fut_vecs
joint_counts

Using the joint counts, we can divide each row (which corresponds to a particular past word) by the number of times that past word was observed. This will give a matrix of probability distributions: the probability of seeing a given future word conditioned on the past word.

In [ ]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

p_counts = joint_counts.sum(axis=1)
morphs = (np.diag((p_counts**(-1)))@joint_counts)
print(morphs)

Voila! We have a matrix containing the predictive states: $\mathrm{Pr}(x_1,\dots,x_{L}|x_0,\dots,x_{-L})$. We could have gotten this faster using tools in `ruckus`. The procedure we just used, of taking the joint count matrix and dividing by the marginal counts, is a special case of a more general formula. Let $\Phi_P$ be the embedding matrix of the past vectors and let $\Phi_F$ be the matrix of the future vectors. The joint count matrix is written as

$$
C_{PF} = \sum_t \Phi_P^\top \Phi_F
$$

and our formula for dividing by marginal counts can be expressed as

$$
C_{F|P} =\left(\sum_t \Phi_P^\top \Phi_P\right)^{-1} \sum_t \Phi_P^\top \Phi_F
$$

where the inverse used is the *matrix inverse*. Just see for yourself:

In [ ]:
from scipy.linalg import inv
past_squared = past_vecs.T@past_vecs
new_morphs = inv(past_squared)@joint_counts
print(new_morphs)

Exactly the same as before! This works because of the structure of one-hot embeddings. The remarkable thing is that this method *still* works for other embeddings, so long as they are capable of uniquely embedding distributions. This applies to the default settings on `ruckus.EigenRKHS` and `ruckus.RandomFourierRBF`. Because of this ubiquitaous property, the `ProductRKHS` object has a built-in `conditional` method for handling this. It takes as arguments the indices that you are conditioning on (`predictor_inds`), the indices to be conditioned (`response_inds`), and a regression solver - because that formula above is the solution to a linear regression problem:

$$
C_{F|P} = \underset{A}{\mathrm{argmin}} \left\|\Phi_F - A\Phi_P\right\|^2
$$

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression(fit_intercept=False)
prod_space.conditional([0],[1],regressor=regressor)
print(regressor.coef_.T)

In the non-discrete setting, where we use embeddings that are not one-hot, we will jump straight to this method instead.

## Finding dimensionality with cross-validation

In addition to the `conditional` method for product RKHS's, `ruckus` also offers a wrapper class, `ConditionalMapWrapper`, which takes a `ProductRKHS` and a `regressor` and bundles them into an object that we can perform cross validation on. This is useful if there are particular parameters that we want to determine.

One of those parameters is the dimensionality of the predictive space. Currently, the output of our predictive states scales exponentially in `L`. However, we know that the data are generated from processes whose predictive states span only a finite number of dimensions---so we ought to be able to reduce the dimensionality. To determine the relevant dimensions of a matrix we can use a singular value decomposition.

In [ ]:
from scipy.linalg import svd

U,s,V = svd(regressor.coef_.T)
plt.scatter(np.arange(len(s)),s)
plt.xlabel('Dimension index')
plt.ylabel('Singular value')
plt.show()

The above printout indicates that only the first few dimensions are needed to capture most of the structure. However, there may not be a very clear cutoff. This is because of sampling error---though the ideal, mathematical predictive space is exactly finite dimensional, sampling error will create spurious structure that inflates the perceived complexity. Adding too much complexity to our model due to sampling error is called "overfitting," and the best medicine to avoid it is cross-validation.

Using the `ConditionalMapWrapper` from `ruckus` and the `GridSearchCV` from `scikit-learn`, we have most of what we need to find an optimal dimension parameter. However, we will need to do one more thing before this is possible: construct a custom regressor that allows us to follow-up the regression with a SVD.

Building custom classes is a good exercise, and thankfully `scikit-learn` provides the necessary tools:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class SVDRegressor(TransformerMixin,BaseEstimator):
    def __init__(self,n_components=2):
        self.n_components = n_components
        
    def fit(self,X,y):
        # Get linear model
        self.regressor_ = LinearRegression(fit_intercept=False).fit(X,y)
        # SVD the linear coefficients
        U,s,VT = svd(self.regressor_.coef_.T)
        # Build a "dimensional bottleneck" by truncating the SVD at n_components dimensions
        self.bottleneck_ = U[:,:self.n_components]@np.diag(s[:self.n_components])@(VT[:self.n_components])
        return self

    def predict(self,X):
        # Apply the bottlenecked linear model
        return X@self.bottleneck_

With this constructed, we can build our `ConditionalMapWrapper` and `GridSearchCV` objects:

In [ ]:
cm_wrapper = ruckus.cv_wrappers.ConditionalMapWrapper(
    prod_space, # Toss in the RKHS
    [0],[1],    # Specify the predictor and response indices
)

In [ ]:
from sklearn.model_selection import GridSearchCV

components_grid = [2,3,4,5]

cm_gridsearch = GridSearchCV(
    cm_wrapper,
    param_grid=dict(
        regressor = [SVDRegressor(n_components=k,) for k in components_grid ]
    ),
    cv=5,
    verbose=3       # This term determines printed outputs during the cross-validation
)

Now we run it!

In [ ]:
cm_gridsearch.fit(prod_space.X_fit_);

To visualize our results, let's make a plot from the cross-validation mean scores with error bars indicating the standard deviations:

In [ ]:
scores = cm_gridsearch.cv_results_['mean_test_score']
errors = cm_gridsearch.cv_results_['std_test_score']
plt.scatter(
    components_grid,1-scores,
)
plt.errorbar(
    components_grid,1-scores,
    yerr=errors,
    capsize=10,
    fmt='none',
    c='k'
)
plt.xlabel('Number of dimensions')
plt.ylabel('Loss function')
plt.xticks(components_grid)

plt.show()

Interpreting cross-validation results can be somewhat nuanced. The "Best Parameters" determined by `GridSearchCV` are just those that have the lowest mean score. However, the error bars often tell a different story. The standard practice is to consider equivalent any scores that are within a standard deviation of one another, and choose the most *conservative* of the options which are equivalent to the minimum. In this case, the most conservative parameter choice is the lowest dimension. Refer back to our original list of the HMM options: does this graph match your knowlege of the process you chose?

**BONUS 1**: Try toggling `N` (the number of samples) and `L` (the length of the window) and see how that influences the cross-validation results.

**BONUS 2**: In `tutorial_utils` there is a method, `gen_anbn`, which generates a special process. Import it and use it to generate `X` and then run this notebook on it. Can you see why it is special?